<a href="https://colab.research.google.com/github/avy666/LLM_Sber/blob/main/home_task_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Решение задачи Question Answering

**Цель домашнего задания**: решить с помощью bert-подобной модели задачу question answering (QA).

В теме №2 мы разобрали две самых популярных архитектуры на базе трансформера: ***BERT и GPT***. Изучили их основные особенности и дальнейшее развитие. Давайте научимся теперь дообучать подобные модели для различных задач. Например, для задачи поиска ответа на вопрос. В целом, любая хорошо обученная gpt-подобная модель с этим справится, но мы хотим обучить bert-подобную модель, чтобы убедиться, что для решения подобной задачи достаточно только энкодеров.

### Подготовка данных

В качестве академического бенчмарка для задачи QA чаще всего используется датасет SQuAD, состоящем из вопросов, заданных краудворкерами по набору статей Википедии, поэтому мы будем использовать именно его.

In [4]:
!pip install transformers datasets -U -qqq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 88.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 20.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is 

In [6]:
from datasets import load_dataset

raw_datasets = load_dataset("squad")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.62k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

Мы можем взглянуть на этот объект, чтобы узнать больше о датасете SQuAD:

In [7]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [8]:
raw_datasets['train'][1]

{'id': '5733be284776f4190066117f',
 'title': 'University_of_Notre_Dame',
 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'question': 'What is in front of the Notre Dame Main Building?',
 'answers': {'text': ['a copper statue of Christ'], 'answer_start': [188]}}

Все необходимое содержится в полях context, question и answers, так что давайте выведем их для первого элемента нашего обучающего набора:

In [9]:
print("Context: ", raw_datasets["train"][0]["context"])
print("Question: ", raw_datasets["train"][0]["question"])
print("Answer: ", raw_datasets["train"][0]["answers"])

Context:  Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.
Question:  To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?
Answer:  {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}


### Загрузка токенизатора

В качестве baseline будем использовать легкий дистилированный берт.

In [10]:
from transformers import AutoTokenizer, RobertaTokenizerFast

# Отключаем токенизатор distilbert
# model_checkpoint = "distilbert/distilbert-base-cased"
# tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# Загружаем токенизатор модели RoBERTa

model_checkpoint = "roberta-base"
tokenizer = RobertaTokenizerFast.from_pretrained(model_checkpoint)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

Прверим работу нашего токенизатора:

In [11]:
context = raw_datasets["train"][0]["context"]
question = raw_datasets["train"][0]["question"]

inputs = tokenizer(question, context)
tokenizer.decode(inputs["input_ids"])

'<s>To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?</s></s>Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.</s>'

Нам надо быть готовым к тому, что будут встречаться длинные контексты, а значит, надо будет ввести предварительную обработку.

Чтобы увидеть, как это работает на текущем примере, мы можем ограничить длину до 100 и использовать скользящее окно из 50 токенов. Основные параметры предобработки следующие:

- ***max_length*** для установки максимальной длины (здесь 100)
- ***truncation="only_second"*** для усечения контекста (который находится во второй позиции), когда вопрос с его контекстом слишком длинный
- ***stride*** для задания количества перекрывающихся токенов между двумя последовательными фрагментами (здесь 50)
- ***return_overflowing_tokens=True***, чтобы сообщить токенизатору, что нам нужны токены переполнения (overflowing tokens)

In [12]:
inputs = tokenizer(
    question,
    context,
    max_length=100,
    truncation="only_second",
    stride=50,
    return_overflowing_tokens=True
)

for ids in inputs["input_ids"]:
    print(tokenizer.decode(ids))

<s>To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?</s></s>Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica</s>
<s>To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?</s></s> in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the</s>
<s>To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?</s></s>nes". Next to the Main Building is the

Как мы можем видеть, наш пример был разбит на четыре части, каждый из которых содержит вопрос и часть контекста. Обратите внимание, что ответ на вопрос (“Bernadette Soubirous”) появляется только в третьей и последней части, поэтому, работая с длинными контекстами таким образом, мы невольно создадим несколько обучающих примеров, в которых ответ не будет включен в контекст. Для этих примеров метками будут start_position = end_position = 0 (таким образом мы предсказываем токен [CLS]). Мы также зададим эти метки в неудачном случае, когда ответ был усечен, то есть у нас будут только его начало (или конец). Для примеров, где ответ полностью находится в контексте, метками будут индекс токена, с которого начинается ответ, и индекс токена, на котором ответ заканчивается.

Датасет предоставляет нам начальный символ ответа в контексте, а прибавив к нему длину ответа, мы можем найти конечный символ в контексте. Чтобы сопоставить их с индексами токенов, нам нужно использовать сопоставление смещений, поэтому добавим в токенизатор еще ***return_offsets_mapping=True***:

In [13]:
inputs = tokenizer(
    question,
    context,
    max_length=100,
    truncation="only_second",
    stride=50,
    return_overflowing_tokens=True,
    return_offsets_mapping=True
)
inputs.keys()

dict_keys(['input_ids', 'attention_mask', 'offset_mapping', 'overflow_to_sample_mapping'])

### Подготовка датасета

Теперь, когда мы шаг за шагом разобрались с предварительной обработкой обучающих данных, мы можем сгруппировать их в функцию, которую будем применять ко всему датасету.

In [14]:
max_length = 384
stride = 128


def preprocess_training_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length"
    )

    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Найдём начало и конец контекста
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # Если ответ не полностью находится внутри контекста, меткой будет (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # В противном случае это начальная и конечная позиции токенов
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

Обратите внимание, что мы ввели две константы для определения максимальной длины и длины скользящего окна, а также добавили немного очистки перед токенизацией: некоторые вопросы в датасете SQuAD имеют лишние пробелы в начале и конце, которые ничего не добавляют (и занимают место при токенизации, если вы используете модель вроде RoBERTa), поэтому мы удалили эти лишние пробелы.

Чтобы применить эту функцию ко всему обучающему набору, мы используем метод Dataset.map() с флагом batched=True.

In [15]:
train_dataset = raw_datasets["train"].map(
    preprocess_training_examples,
    batched=True,
    remove_columns=raw_datasets["train"].column_names
)

Map:   0%|          | 0/87599 [00:00<?, ? examples/s]

### Обработка валидационных данных

Совершенно аналогично выполним предобработку валидационных данных.

In [16]:
def preprocess_validation_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length"
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

In [17]:
validation_dataset = raw_datasets["validation"].map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=raw_datasets["validation"].column_names
)

Map:   0%|          | 0/10570 [00:00<?, ? examples/s]

### Дообучение модели

Кажется, у нас все готово для дообучения. Осталось только определить метрику качества. Нам нужно правильно угадать два токена (токен начала ответа и токен конца ответа). Помимо этого, надо чтобы все символы внутри ответа как можно ближе соответствовали символам в ответе. Для этого придумали две метрики: ***F1*** и ***exact match***.

F1 метрика для текста вычисляется по отдельным словам в прогнозе по сравнению со словами в истинном ответе. Количество общих слов между прогнозом и истинным ответом является основой оценки F1: точность — это отношение количества общих слов к общему количеству слов в прогнозе , а полнота — это отношение количества общих слов к общему количеству слов в истинном ответе.

$$ F_{1}=2\dfrac{precision*recall}{precision+recall}$$

Exact match более жесткая метрика, так как устроена она по принципу "все или ничего". Для того, чтобы она была равна 1 для одного примера необходимо полное совпадение всех символов в ответе, иначе она равна сразу 0.

Давайте реализуем теперь эти две метрики. Для начала нам надо нормализовать текст, поскольку в зависимости от модели, могут генерироваться разные посторонние символы и сам ответ может быть как с большой, так и с маленькой буквы. Чтобы избежать всего этого и проводят простую нормализацию.

In [18]:
def normalize_text(s):
    """
    Функция для нормализации текста. Удаляет артикли, пунктуацию и приводит текст к единому виду,
    устраняя лишние пробелы и приводя все символы к нижнему регистру.

    Args:
    s (str): Входной текст.

    Returns:
    str: Нормализованный текст.
    """
    # Ваш код здесь
    import string

    # Переводим весь текст в нижний регистр
    text = s.lower()

    # Создаем список стоп-слов (артиклей)
    stop_words = ["a", "an", "the"]

    # Разделяем текст на отдельные слова
    words = text.split()

    # Фильтруем слова, убирая артикли и знаки пунктуации
    filtered_words = []
    for word in words:
        if word not in stop_words and all(char not in string.punctuation for char in word):
            filtered_words.append(word)

    # Собираем нормализованный текст без лишних пробелов
    normalized_text = ' '.join(filtered_words)

    return normalized_text

In [19]:
def compute_exact_match(prediction, truth):
    """
    Функция для вычисления точного совпадения между предсказанием и истинным значением.

    Args:
    prediction (str): Предсказанный текст.
    truth (str): Истинный текст.

    Returns:
    int: 1, если нормализованный предсказанный текст совпадает с нормализованным истинным текстом, иначе 0.
    """
    # Ваш код здесь
    if normalize_text(prediction) == normalize_text(truth):
        return 1
    else:
        return 0

In [20]:
def compute_f1(prediction, truth):
    """
    Функция для вычисления F1-меры между предсказанным текстом и истинным текстом.

    Args:
    prediction (str): Предсказанный текст.
    truth (str): Истинный текст.

    Returns:
    float: Значение F1-меры, показывающее гармоническое среднее между точностью и полнотой.
    """
    # Ваш код здесь
    # Преобразуем строки в списки слов
    pred_tokens = normalize_text(prediction).split()
    true_tokens = normalize_text(truth).split()

    # Подсчитываем количество уникальных слов в каждом списке
    pred_counter = Counter(pred_tokens)
    true_counter = Counter(true_tokens)

    # Находим пересечение множеств слов
    intersection_count = sum((pred_counter & true_counter).values())

    # Вычисляем точность (precision)
    precision = intersection_count / len(pred_tokens) if len(pred_tokens) > 0 else 0

    # Вычисляем полноту (recall)
    recall = intersection_count / len(true_tokens) if len(true_tokens) > 0 else 0

    # Вычисляем F1-метрику
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return f1_score

In [21]:
!pip install evaluate accelerate -U -qqq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 89.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 72.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 55.0 MB/s eta 0:00:00


Если не хочется реализовывать подсчет метрик самостоятельно, можно воспользоваться библиотекой evaluate.

In [22]:
import evaluate

metric = evaluate.load("squad")

In [23]:
from tqdm.auto import tqdm
import collections
import numpy as np

def compute_metrics(start_logits, end_logits, features, examples, n_best, max_answer_length):
    example_to_features = collections.defaultdict(list)
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []
    for example in tqdm(examples):
        example_id = example["id"]
        context = example["context"]
        answers = []

        # Итерируемся по всем ответам, ассоциированным с этим примером
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Пропускаем ответы, которые не полностью соответствуют контексту
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Пропускайте ответы, длина которых либо < 0, либо > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)

        # Выбираем ответ с лучшей оценкой
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(
                {"id": example_id, "prediction_text": best_answer["text"]}
            )
        else:
            predicted_answers.append({"id": example_id, "prediction_text": ""})

    theoretical_answers = [{"id": ex["id"], "answers": ex["answers"]} for ex in examples]
    return metric.compute(predictions=predicted_answers, references=theoretical_answers)

Загружаем модель и начинаем дообучение!

In [24]:
# Меняем используемую модель на RoBERTa

# from transformers import AutoModelForQuestionAnswering
from transformers import RobertaForQuestionAnswering, RobertaConfig

config = RobertaConfig.from_pretrained(model_checkpoint)
model = RobertaForQuestionAnswering.from_pretrained(model_checkpoint, config=config)

# model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForQuestionAnswering were not initialized from the model checkpoint at roberta-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Будем использовать встроенный класс Trainer для удобства.

In [ ]:
# Пропускаем блок, в нем используется предыдущая модель
from transformers import TrainingArguments

args = TrainingArguments(
    "bert-finetuned-squad",
    evaluation_strategy="no",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    fp16=True,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=128
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Пропускаем блок

from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    tokenizer=tokenizer
)
trainer.train()

<ipython-input-48-25297bb3565e>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: avy666 (avy666-iplc) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Step,Training Loss
500,2.579000
1000,1.595700


Step,Training Loss
500,2.579000
1000,1.595700
1500,1.369700
2000,1.215800
2500,1.170300
3000,1.094600
3500,1.009100
4000,1.017400


TrainOutput(global_step=4161, training_loss=1.3667860922690798, metrics={'train_runtime': 2689.6806, 'train_samples_per_second': 98.966, 'train_steps_per_second': 1.547, 'total_flos': 2.608361755366349e+16, 'train_loss': 1.3667860922690798, 'epoch': 3.0})

Когда обучение завершено, мы можем наконец оценить нашу модель. Метод predict() класса Trainer вернет кортеж, где первыми элементами будут предсказания модели (здесь пара с начальным и конечным логитами). Мы отправляем его в нашу функцию compute_metrics():

In [ ]:
# Пропускаем этот расчет метрик
predictions, _, _ = trainer.predict(validation_dataset)
start_logits, end_logits = predictions
compute_metrics(start_logits, end_logits, validation_dataset, raw_datasets["validation"], n_best=20, max_answer_length=100)

  0%|          | 0/10570 [00:00<?, ?it/s]

{'exact_match': 73.68968779564806, 'f1': 82.66943560654688}

In [ ]:
start_logits

array([[-3.4960938, -8.703125 , -8.8984375, ..., -9.5546875, -9.5625   ,
        -9.546875 ],
       [-3.4570312, -8.71875  , -8.921875 , ..., -9.5546875, -9.5546875,
        -9.546875 ],
       [-4.1484375, -8.40625  , -8.1640625, ..., -9.4609375, -9.4453125,
        -9.4453125],
       ...,
       [-1.5400391, -8.84375  , -8.9375   , ..., -9.5703125, -9.6015625,
        -9.5859375],
       [-0.5048828, -8.78125  , -8.953125 , ..., -9.6796875, -9.609375 ,
        -9.6328125],
       [-1.3007812, -8.6875   , -8.734375 , ..., -9.6328125, -9.6796875,
        -9.65625  ]], dtype=float32)

Отлично! Получились хорошие значения метрик. Попробуйте теперь получить exact_match больше 85% и f1 больше 90%. Используйте разные модели, разные гиперпараметры и тд.

In [2]:
# Ваш код здесь
# Используем модель RoBERTa
from transformers import TrainingArguments

args = TrainingArguments(
    "roberta-finetuned-squad",
    evaluation_strategy="no",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    fp16=True,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=128
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [3]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    tokenizer=tokenizer
)
trainer.train()

NameError: name 'model' is not defined

In [1]:
trainer.save_model(output_dir="robrerta_model")

NameError: name 'trainer' is not defined

In [ ]:
model.load_state_dict(torch.load("robrerta_model.pt"))

Проверьте на любом примере, что написанные вами метрики совпадают по значению с метриками из evaluate.

In [ ]:
# Считаем метрики для новой модели
predictions, _, _ = trainer.predict(validation_dataset)
start_logits, end_logits = predictions
compute_metrics(start_logits, end_logits, validation_dataset, raw_datasets["validation"], n_best=20, max_answer_length=100)

  0%|          | 0/10570 [00:00<?, ?it/s]

{'exact_match': 85.3263954588458, 'f1': 91.78690160203536}

In [1]:
start_logits

NameError: name 'start_logits' is not defined

In [ ]:
# Ваш код здесь